In [1]:
# save_best_model_video_fix.py
# Save a gameplay video using your best A2C model (fixed step/unpack for VecEnv).

import os
import gymnasium as gym
import gymnasium_super_mario_bros  # registers env
import numpy as np
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import VecVideoRecorder

# Make sure your custom CNN is importable when loading the model
from mario_cnn_a2c_2 import make_mario_env, NatureCNN  # noqa: F401

# ---- paths (adjust exp index if needed) ----
EXP_DIR   = "results/a2c/exp7"
MODEL_DIR = os.path.join(EXP_DIR, "models")
BEST_ZIP  = os.path.join(MODEL_DIR, "best_model", "best_model.zip")
VIDEO_DIR = os.path.join(EXP_DIR, "videos")
os.makedirs(VIDEO_DIR, exist_ok=True)

# ---- load model ----
model = A2C.load(BEST_ZIP)

# ---- make a 1-env VecEnv suitable for recording ----
env = make_mario_env(
    env_id="SuperMarioBros-1-1-v0",
    n_envs=1,
    seed=42,
    wrapper_kwargs={"frame_skip": 4, "screen_size": 84},
    use_vec_normalize=False,                 # simpler for video export
    env_kwargs={"render_mode": "rgb_array"}, # required to capture frames
)

# Record one long episode
video_length = 10_000
env = VecVideoRecorder(
    env,
    VIDEO_DIR,
    record_video_trigger=lambda step: step == 0,  # record first episode
    video_length=video_length,
    name_prefix="mario_A2C_best",
    #fps=15,  # ↓ slower playback
)
# ---- run one episode (VecEnv returns 4 values) ----
obs = env.reset()  # VecEnv.reset() -> obs (no info)
done = False
t = 0

while not done and t < video_length:
    action, _ = model.predict(obs, deterministic=True)
    # VecEnv.step() -> obs, rewards, dones, infos   (NOT terminated/truncated)
    obs, rewards, dones, infos = env.step(action)
    done = bool(dones[0])
    t += 1

env.close()
print(f"Saved video(s) to: {VIDEO_DIR}")


ModuleNotFoundError: No module named 'gymnasium'

'/home/sagemaker-user/gymnasium-marioV1'